### A notebook to calculate critical temperature using CLT and smoothed Binder cumulant data

In [2]:
#imports 
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy import optimize
import json

In [3]:
def initLattice(latticeSize: int, hot: bool) -> list[list[int]]:
    lattice = np.zeros((latticeSize, latticeSize))
    if hot:
        for i in range(latticeSize):
            for j in range(latticeSize):
                #pick a random spin
                spin = 0
                randomInt = random.randint(0,1)
                if(randomInt == 1):
                    spin = 1
                else:
                    spin = -1
                #set lattice site equal to the random spin    
                lattice[i][j] = spin
    else: #lattice is cold
        for i in range(latticeSize):
            for j in range(latticeSize):
                #set all lattice sites to spin up
                spin = 1
                lattice[i][j] = spin
    
    return lattice

In [4]:
# Calculate change in energy of lattice by flipping a single site (i,j)
def deltaU(i: int, j: int, lattice: list) -> float:
    '''
    This calulation requires considering neighboring sites (first term in Hamiltonian)
    Therefore, we will use periodic boundary conditions (torus)
    I would like to imlement the external field term so you can drive the system to specific states

    E1 = -spin(i,j)*sum(spin(neighbors)),     E2 = spin(i,j)*sum(spin(neighbors))
    Ediff = E2 - E1 = 2spin(i,j)*sum(neighbors) (if spin(i,j) is 1 (up))            <<<<< NO epsilon/J? unclear why, currently just implementing pseudocode exactly as written

    In the mean field approximation E_up = -4J*sum(spin(neighbors))/4) 

    i is vertical, j is horizontal, zero indexed
    '''

    size = len(lattice)
    # If site is in an edge, apply periodic boundary conditions
    if(i == 0):
        top = lattice[size-1,j]
    else:
        top = lattice[i-1][j]
    if(i == size-1):
        bottom = lattice[0][j]
    else:
        bottom = lattice[i+1][j]
    if(j == 0):
        left = lattice[i][size-1]
    else:
        left = lattice[i][j-1]
    if(j == size-1):
        right = lattice[i][0]
    else:
        right = lattice[i][j+1]

    #now calculate the energy difference
    Ediff = 2*lattice[i][j]*(top+bottom+left+right)
    return Ediff

In [5]:
#define a weird cumulant and determine when the intersect to find critical temperature.
def m24(lattice, temp, iterations):
    m = []
    for iteration in range(iterations):

        if((iteration % 100 == 0) and (iteration != 0)):
            sum = 0
            for xSite in range(lattice[0].size):
                for ySite in range(lattice[0].size):
                    sum += lattice[xSite][ySite]
            m.append(abs(sum)) #NEW: try storing abs of mag

        i = random.randint(0,lattice[0].size-1)
        j = random.randint(0,lattice[0].size-1)
        Ediff = deltaU(i,j,lattice)
        #Metropolis to decide whether site should be flipped. Needs to be iterated 100 times??
        if(Ediff <= 0):
            lattice[i][j] = -lattice[i][j] 
        else:
            #now only flip site according to Boltzmann factor
            boltzmannRandom = random.uniform(0,1)
            if(boltzmannRandom < np.exp(-Ediff/temp)): #Ediff must be positive so exponential is between 0 and 1
                lattice[i][j] = -lattice[i][j]


    return (np.power(m,2), np.power(m,4))

In [6]:
def calcCritTemp():
    '''
    Calculate the crtical temperature in the 2d square lattice Ising model using intersection of Binder cumulants.
    '''

    #obtain necessairy quantities for lattice 1
    avgm210 = []
    avgm410 = []
    size = 24
    lat = initLattice(size,False)

    tempRange = np.linspace(2.1,2.4,1000)
    for temp in tempRange:
        percent = round(((temp-2.1) / (2.4-2.1)) * 100, 1)
        #print("T = %s      %s%%      s=%s" % (round(temp,3), percent, size))
        m10 = m24(lat, temp, 100000) #lattice size 20 mags
        m210 = m10[0] #m^2 for lattice size 20
        m410 = m10[1] #m^4 for lattice size 20
        
        #average m^2
        avg210 = np.average(m210)  # get rid of normalization  / (np.power(size,2)) / (np.power(size,2))
        avgm210.append(avg210) 

        #average m^4
        avg410 = np.average(m410)  # get rid of normalization  / (np.power(size,2))/ (np.power(size,2))
        avgm410.append(avg410)

    #obtain necesairy quantiites for lattice 2
    avgm220 = []
    avgm420 = []
    size = 18
    lat = initLattice(size,False)

    tempRange = np.linspace(2.1,2.4,1000)
    for temp in tempRange:
        percent = round(((temp-2.1) / (2.4-2.1)) * 100, 1)
        #print("T = %s      %s%%      s=%s" % (round(temp,3), percent, size))
        m20 = m24(lat, temp, 100000) #lattice size 20 mags #500000
        m220 = m20[0] #m^2 for lattice size 20
        m420 = m20[1] #m^4 for lattice size 20
        
        #average m^2
        avg220 = np.average(m220)  # get rid of normalization  / (np.power(size,2))   SHOULD I ADD ERROR BARS HERE?
        avgm220.append(avg220) 

        #average m^4
        avg420 = np.average(m420)  # get rid of normalization/ (np.power(size,2))
        avgm420.append(avg420) 

    #Create Binder cumulant arrays
    div10 = []
    for i in range(len(avgm210)):
        div10.append(1 - (avgm410[i]) / (3*np.power(avgm210[i],2)) )

    div20 = []
    for i in range(len(avgm220)):
        div20.append(1 - (avgm420[i]) / (3*np.power(avgm220[i],2)) )


    #smooth the cumulants by averaging
    averagingWindow = 200

    avg10 = []
    for i in range(len(div10)+1):
        if(i % averagingWindow == 0 and i != 0):
            avg10.append(np.average(div10[i-averagingWindow:i]))

    avg20 = []
    for i in range(len(div20)+1):
        if(i % averagingWindow == 0 and i != 0):
            avg20.append(np.average(div20[i-averagingWindow:i]))

    #interpolate and calculate intersection by findinger zero using Brent's method (brentq)
    xData = np.linspace(2.1,2.4,int((1000/averagingWindow)))
    f10 = interpolate.interp1d(xData, avg10)
    f20 = interpolate.interp1d(xData, avg20)
    def fRoot(x):
        return f20(x)-f10(x)
   
    #if intersection found, return it, otherwise return 0
    if(fRoot(2.1)*fRoot(2.4) < 0):
        Tc = optimize.brentq(fRoot, 2.1, 2.4)
    else:
        Tc = 0
    return Tc

In [7]:
critTempArr = []
iters = 40

for i in range(iters):
    percent = (i/iters) * 100
    print("%s%%" % percent)
    critTemp = calcCritTemp()
    critTempArr.append(critTemp)
    

with open('CriticalTemperatureOutput', 'w') as filehandle:
    json.dump(critTempArr, filehandle)

print("100.0%%")

0.0%
2.5%
5.0%
7.5%
10.0%
12.5%
15.0%
17.5%
20.0%
22.5%
25.0%
27.500000000000004%
30.0%
32.5%
35.0%
37.5%
40.0%
42.5%
45.0%
47.5%
50.0%
52.5%
55.00000000000001%
57.49999999999999%
60.0%
62.5%
65.0%
67.5%
70.0%
72.5%
75.0%
77.5%
80.0%
82.5%
85.0%
87.5%
90.0%
92.5%
95.0%
97.5%
100.0%%


In [10]:
nonZeroArr = []
for i in critTempArr:
    if i != 0:
        nonZeroArr.append(i)

np.average(nonZeroArr)

2.2977432026533577